In [1]:
from pygrok import Grok

In [2]:
pattern1 = r"%{MONTH:month} %{MONTHDAY:day} %{YEAR:year} %{TIME:time} %{DATA:host} %%%{NUMBER:versionNumber}%{DATA:moduleName}/%{NUMBER:severity}/%{WORD:logType}%{GREEDYDATA:description}"

In [3]:
grok = Grok(pattern1)

In [4]:
log_entry1= "Jun 11 2024 11:52:42 GPN-0003_TRC-CR_X8-01 %%01OSPF/4/CONFLICT_ROUTERID_INTF(l):CID=0x8082047e;OSPF router ID conflict is detected on the interface.(ProcessId=801, RouterId=10.0.0.10, AreaId=0.0.0.0, InterfaceName=GigabitEthernet6/0/11.801, IpAddr=10.0.0.22, PacketSrcIp=10.0.0.10)"

In [5]:
matching = grok.match(log_entry1)

In [6]:
print(matching)

{'month': 'Jun', 'day': '11', 'year': '2024', 'time': '11:52:42', 'host': 'GPN-0003_TRC-CR_X8-01', 'versionNumber': '01', 'moduleName': 'OSPF', 'severity': '4', 'logType': 'CONFLICT_ROUTERID_INTF', 'description': '(l):CID=0x8082047e;OSPF router ID conflict is detected on the interface.(ProcessId=801, RouterId=10.0.0.10, AreaId=0.0.0.0, InterfaceName=GigabitEthernet6/0/11.801, IpAddr=10.0.0.22, PacketSrcIp=10.0.0.10)'}


In [7]:
print(type(matching))

<class 'dict'>


In [8]:
regex_pattern1 = r"(?P<vendor>[A-Z]+)-(?P<physical_site_id>\d+)_(?P<geolocation_code>[A-Z\d]+)-(?P<router_type>[A-Z\d]+)_(?P<router_model_number>[A-Z\d]+)-(?P<router_importance>[A-Z\d]+) (?P<vendor_identifier>%%)(?P<version_number>\d+)(?P<module_name>[A-Z]+)/(?P<severity>\d+)/(?P<log_type>[A-Z_]+)(?P<log_status>[\(\)a-z]+):CID=(?P<cid>[0-9a-zA-Z]+);(?P<description>.+)"


In [14]:
from datetime import datetime

In [15]:
def extract_time_info(message):
    # Extract timestamp
    timestamp_str = message.split(' ')[0:4]
    timestamp_str = ' '.join(timestamp_str)
    dt = datetime.strptime(timestamp_str, "%b %d %Y %H:%M:%S")
    year = dt.year
    month = dt.month
    day_of_month = dt.day
    hour = dt.hour
    minute = dt.minute
    second = dt.second
    millisecond = dt.microsecond // 1000
    day_of_week = dt.weekday()  # Monday = 0, Sunday = 6
    day_of_year = dt.timetuple().tm_yday
    week_of_year = dt.isocalendar()[1]
    is_weekend = int(dt.weekday() >= 5)
    seconds_since_midnight = hour * 3600 + minute * 60 + second
    time_of_day_bucket = 'Morning' if hour < 12 else 'Afternoon' if hour < 18 else 'Evening' if hour < 21 else 'Night'
    time_dict = {
        'timestamp': timestamp_str,
        'year': year,
        'month': month,
        'day_of_month': day_of_month,
        'hour': hour,
        'minute': minute,
        'second': second,
        'millisecond': millisecond,
        'day_of_week': day_of_week,
        'day_of_year': day_of_year,
        'week_of_year': week_of_year,
        'is_weekend': is_weekend,
        'seconds_since_midnight': seconds_since_midnight,
        'time_of_day_bucket': time_of_day_bucket
    }
    return time_dict

In [16]:
def remove_time_info_from_message(message):
    # Find the position of the first colon
    first_colon_pos = message.find(':')

    # Find the position of the second colon by starting the search after the first colon
    second_colon_pos = message.find(':', first_colon_pos + 1)

    # Slice the string two characters after the second colon
    truncated_message = message[second_colon_pos + 4:]

    # Print the truncated message
    return truncated_message

In [62]:
# raw_message = 'Jun 11 2024 11:53:06 GPN-0277_DAV-CLI_S57-01 %%01SNMP/4/SNMP_FAIL(l)[66039]:Failed to login through SNMP. (Ip=172.31.251.25, Times=3, Reason=decoded PDU error)'
raw_message = 'Jun 11 2024 11:52:55 GPN-2623_PRO-ART-APL-PAR_S57-01 %%01SSH/5/SSH_CONNECT_CLOSED(s)[25804]:SSH connect was closed. (IP=172.31.251.24, VPNInstanceName= , UserName=)'

In [63]:
time_dict = extract_time_info(message=raw_message)

In [64]:
time_dict

{'timestamp': 'Jun 11 2024 11:52:55',
 'year': 2024,
 'month': 6,
 'day_of_month': 11,
 'hour': 11,
 'minute': 52,
 'second': 55,
 'millisecond': 0,
 'day_of_week': 1,
 'day_of_year': 163,
 'week_of_year': 24,
 'is_weekend': 0,
 'seconds_since_midnight': 42775,
 'time_of_day_bucket': 'Morning'}

In [65]:
truncated_message = remove_time_info_from_message(message=raw_message)

In [66]:
truncated_message

'GPN-2623_PRO-ART-APL-PAR_S57-01 %%01SSH/5/SSH_CONNECT_CLOSED(s)[25804]:SSH connect was closed. (IP=172.31.251.24, VPNInstanceName= , UserName=)'

In [67]:
# regex_pattern2 = r"(?P<vendor>[A-Z]+)-(?P<physical_site_id>\d+)_(?P<geolocation_code>[A-Z\d]+)-(?P<router_type>[A-Z\d]+)_(?P<router_model_number>[A-Z\d]+)-(?P<router_importance>[A-Z\d]+) (?P<vendor_identifier>%%)(?P<version_number>\d+)(?P<module_name>[A-Z]+)/(?P<severity>\d+)/(?P<log_type>[A-Z_]+)(?P<log_status>[\(\)a-z]+)\[(?P<error_code>\d+)\]:(?P<description>.+)\(Ip=(?P<ip>\d{1,3}(?:\.\d{1,3}){3}), Times=(?P<times>\d+), Reason=(?P<reason>.+)\)"
regex_pattern2 = r"(?P<vendor>[A-Z]+)-(?P<physical_site_id>\d+)_(?P<geolocation_code>[A-Z\d]+)-(?P<router_type>[A-Z\d]+)_(?P<router_model_number>[A-Z\d]+)-(?P<router_importance>[A-Z\d]+) (?P<vendor_identifier>%%)(?P<version_number>\d+)(?P<module_name>[A-Z]+)/(?P<severity>\d+)/(?P<log_type>[A-Z_]+)(?P<log_status>[\(\)a-z]+)\[(?P<sequence_number>\d+)\]:(?P<description>.+)"

In [75]:
import re

matching = re.search(regex_pattern2, truncated_message)

In [76]:
matching

In [77]:
print(matching)

None


In [70]:
matching.groupdict()

AttributeError: 'NoneType' object has no attribute 'groupdict'

In [71]:
def extract_metadata_from_description(description):
    if description:
        opening_bracket_pos = description.find('(')
        if opening_bracket_pos >= 0:
            
            # Slice the string and separate metadata
            metadata = description[opening_bracket_pos + 1:-1]

            
        else:
            print("no metadata available inside the description")
            metadata = "" 
    else:
        print("no description and hence no metadata")
        metadata = ""
        
    return metadata    


In [72]:
extracted_info = matching.groupdict()
description = extracted_info.get("description")
metadata = extract_metadata_from_description(description=description)

AttributeError: 'NoneType' object has no attribute 'groupdict'

In [34]:
metadata

'Ip=172.31.251.25, Times=3, Reason=decoded PDU error'

In [35]:
raw_message2 = 'Jun 11 2024 11:52:54 GPN-0866_ELA-CLI_S57-01 LLDP/4/RATEEXCESSIVE:OID 1.3.6.1.4.1.2011.5.25.134.2.7 The rate of received PDUs exceeded the threshold. (IfName=GigabitEthernet0/0/5)'


In [36]:
time_dict2 = extract_time_info(message=raw_message2)

In [37]:
time_dict2

{'timestamp': 'Jun 11 2024 11:52:54',
 'year': 2024,
 'month': 6,
 'day_of_month': 11,
 'hour': 11,
 'minute': 52,
 'second': 54,
 'millisecond': 0,
 'day_of_week': 1,
 'day_of_year': 163,
 'week_of_year': 24,
 'is_weekend': 0,
 'seconds_since_midnight': 42774,
 'time_of_day_bucket': 'Morning'}

In [42]:
truncated_message2 = remove_time_info_from_message(message=raw_message2)

In [43]:
truncated_message2

'GPN-0866_ELA-CLI_S57-01 LLDP/4/RATEEXCESSIVE:OID 1.3.6.1.4.1.2011.5.25.134.2.7 The rate of received PDUs exceeded the threshold. (IfName=GigabitEthernet0/0/5)'

In [48]:
regex_pattern3 = r"(?P<vendor>[A-Z]+)-(?P<physical_site_id>\d+)_(?P<geolocation_code>[A-Z\d]+)-(?P<router_type>[A-Z\d]+)_(?P<router_model_number>[A-Z\d]+)-(?P<router_importance>[A-Z\d]+) (?P<vendor_identifier>[A-Z]+)\/(?P<severity>\d+)\/(?P<log_type>[A-Z_]+):OID (?P<oid>[A-Z\d\s.]+) (?P<description>.+)"

In [49]:

matching2 = re.search(regex_pattern3, truncated_message2)

In [50]:
matching2

<re.Match object; span=(0, 158), match='GPN-0866_ELA-CLI_S57-01 LLDP/4/RATEEXCESSIVE:OID >

In [51]:
matching2.groupdict()

{'vendor': 'GPN',
 'physical_site_id': '0866',
 'geolocation_code': 'ELA',
 'router_type': 'CLI',
 'router_model_number': 'S57',
 'router_importance': '01',
 'vendor_identifier': 'LLDP',
 'severity': '4',
 'log_type': 'RATEEXCESSIVE',
 'oid': '1.3.6.1.4.1.2011.5.25.134.2.7',
 'description': 'The rate of received PDUs exceeded the threshold. (IfName=GigabitEthernet0/0/5)'}

In [78]:
msg = 'hellog user failed'

In [84]:
if 'faild' or 'fail' in '':
    print("hi")
else:
    print('no')    

hi
